# Generating random samples for many files at once

In [ ]:
from __future__ import annotations
import random
from pathlib import Path
import pyarrow as pa
import pyarrow.parquet as pq

# ----------------- CONFIG -----------------
# Directories containing your input Parquet files
INPUT_DIRS = [
    "/scratch/users/luigi.silva/speczs-catalogs/processed",
    "/scratch/users/luigi.silva/speczs-catalogs/johns-catalogs",
]

# Directory where the sampled files will be saved
OUTPUT_DIR = Path("/scratch/users/luigi.silva/pzserver_pipelines/combine_redshift_dedup/test_data")

# Maximum number of rows per sample
SAMPLE_MAX = 1000

# Random seed for reproducibility
SEED = 42
# ------------------------------------------


def ensure_outdir(path: Path) -> None:
    """Make sure the output directory exists."""
    path.mkdir(parents=True, exist_ok=True)


def sample_parquet_reservoir(in_path: Path, k: int, seed: int = 42) -> pa.Table:
    """
    Perform random sampling (without replacement) from a Parquet file.
    - If the file has <= k rows, return all rows.
    - Otherwise, use reservoir sampling to select k rows while reading in streaming mode.
      This avoids loading the entire file into memory.
    """
    random.seed(seed)

    pf = pq.ParquetFile(str(in_path))
    total_rows = pf.metadata.num_rows

    # Case 1: small file -> just read all rows
    if total_rows <= k:
        return pf.read()

    # Case 2: large file -> reservoir sampling
    schema = pf.schema_arrow
    columns = [[] for _ in schema]  # temporary storage for sampled rows

    seen = 0  # number of rows processed so far

    for batch in pf.iter_batches():
        # Get columns as arrays for easier row access
        cols = [batch.column(i) for i in range(batch.num_columns)]
        n = batch.num_rows

        for i in range(n):
            if seen < k:
                # Fill the reservoir until it reaches size k
                for c_idx, arr in enumerate(cols):
                    columns[c_idx].append(arr[i].as_py())
            else:
                # Replace elements with decreasing probability
                j = random.randint(0, seen)
                if j < k:
                    for c_idx, arr in enumerate(cols):
                        columns[c_idx][j] = arr[i].as_py()
            seen += 1

    # Convert sampled Python lists back to Arrow arrays
    arrays = [pa.array(col, type=schema.field(i).type) for i, col in enumerate(columns)]
    table = pa.Table.from_arrays(arrays, names=[f.name for f in schema])

    assert len(table) == min(k, total_rows)
    return table


# ----------------- MAIN EXECUTION -----------------
ensure_outdir(OUTPUT_DIR)
count = 0

for d in INPUT_DIRS:
    in_dir = Path(d)
    if not in_dir.is_dir():
        print(f"[WARN] Directory not found: {in_dir}")
        continue

    for p in sorted(in_dir.glob("*.parquet")):
        survey_name = p.stem  # file name without extension
        out_path = OUTPUT_DIR / f"{survey_name}_random_sample.parquet"

        print(f"[INFO] Sampling {p} -> {out_path}")
        tbl = sample_parquet_reservoir(p, SAMPLE_MAX, seed=SEED)

        pq.write_table(tbl, out_path)
        count += 1

print(f"[DONE] {count} files processed. Output in: {OUTPUT_DIR}")

# Generating random sample for a single file

In [ ]:
from __future__ import annotations
import random
from pathlib import Path
import pyarrow as pa
import pyarrow.parquet as pq

# --------------- CONFIG ---------------
# Single input Parquet file (extension "parqut" is fine as long as the file exists and is Parquet)
INPUT_FILE = Path("/scratch/users/luigi.silva/pzserver_pipelines/combine_redshift_dedup/process001/crd.parquet")

# Exact output path
OUTPUT_PATH = Path("./test_data/pipeline_generated_sample.parquet")

# Maximum number of rows in the sample
SAMPLE_MAX = 1000

# Random seed for reproducibility
SEED = 42
# --------------------------------------


def sample_parquet_reservoir(in_path: Path, k: int, seed: int = 42) -> pa.Table:
    """
    Return a random sample (without replacement) of up to k rows from a Parquet file.
    - If the file has <= k rows, return the whole file.
    - Otherwise, use reservoir sampling while streaming through the file in row batches
      (avoids loading the entire file into memory).
    """
    random.seed(seed)

    pf = pq.ParquetFile(str(in_path))
    total_rows = pf.metadata.num_rows
    schema = pf.schema_arrow

    # Handle empty files explicitly
    if total_rows == 0:
        arrays = [pa.array([], type=schema.field(i).type) for i in range(len(schema))]
        return pa.Table.from_arrays(arrays, names=[f.name for f in schema])

    # Small file -> just read all rows
    if total_rows <= k:
        return pf.read()

    # Large file -> reservoir sampling
    columns = [[] for _ in range(len(schema))]  # temporary storage for sampled rows
    seen = 0  # number of rows processed so far

    for batch in pf.iter_batches():
        # Access columns as Arrow arrays for quick row access
        cols = [batch.column(i) for i in range(batch.num_columns)]
        n = batch.num_rows

        for i in range(n):
            if seen < k:
                # Fill the reservoir until it reaches size k
                for c_idx, arr in enumerate(cols):
                    columns[c_idx].append(arr[i].as_py())
            else:
                # Replace elements with decreasing probability
                j = random.randint(0, seen)
                if j < k:
                    for c_idx, arr in enumerate(cols):
                        columns[c_idx][j] = arr[i].as_py()
            seen += 1

    # Convert sampled Python lists back to Arrow arrays with the original schema types
    arrays = [pa.array(col, type=schema.field(i).type) for i, col in enumerate(columns)]
    table = pa.Table.from_arrays(arrays, names=[f.name for f in schema])

    assert len(table) == min(k, total_rows), "Sample size mismatch"
    return table


def main():
    if not INPUT_FILE.is_file():
        raise FileNotFoundError(f"Input Parquet not found: {INPUT_FILE}")

    OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)

    print(f"[INFO] Sampling {INPUT_FILE} -> {OUTPUT_PATH}")
    tbl = sample_parquet_reservoir(INPUT_FILE, SAMPLE_MAX, seed=SEED)
    pq.write_table(tbl, OUTPUT_PATH)
    print(f"[DONE] {len(tbl)} rows written to: {OUTPUT_PATH}")


if __name__ == "__main__":
    main()